<a href="https://colab.research.google.com/github/CarlosLeandro09/DataAnalysisRadiology/blob/main/CT_Analisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importar pacotes**

In [ ]:
import pandas as pd
import numpy as np

**Importar CSV do drive**

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CT_chest.csv',sep=';',encoding='iso-8859-1')
df

,"Modality [CS:0008,0060]","Manufacturers Model Name [LO:0008,1090]","Patients Name [PN:0010,0010]","Patient ID [LO:0010,0020]","Patients Sex [CS:0010,0040]","Patients Age [AS:0010,1010]","Patients Size [DS:0010,1020]","Patients Weight [DS:0010,1030]","Body Part Examined [CS:0018,0015]","Scan Options [CS:0018,0022]","Slice Thickness [DS:0018,0050]","KVP [DS:0018,0060]","Table Feed per Rotation [FD:0018,9310]","Total Collimation Width [FD:0018,9307]","CTDIvol [FD:0018,9345]","X-Ray Tube Current [IS:0018,1151]","AcquisitionTime [TM:0008,0032]","Contrast/Bolus Agent [LO:0018,0010]","Slice Location [DS:0020,1041]",filesize (kb),hasPixeldata,path,filename,Unnamed: 23
0,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,55.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img1,NaN
1,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,64.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img10,NaN
2,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,154.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img100,NaN
3,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,155.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img101,NaN
4,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,156.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img102,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10717,CT,Brilliance 10,Anonymous,NaN,F,062Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,7.4,167,81604.0,NaN,136.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img95,NaN
10718,CT,Brilliance 10,Anonymous,NaN,F,062Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,7.4,167,81604.0,NaN,137.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img96,NaN
10719,CT,Brilliance 10,Anonymous,NaN,F,062Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,7.4,167,81604.0,NaN,138.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img97,NaN
10720,CT,Brilliance 10,Anonymous,NaN,F,062Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,7.4,167,81604.0,NaN,139.5,522,True,C:\Users\clean\OneDrive\Área de Trabalho\Image...,img98,NaN


**Limpar dados**

In [15]:
data = df.filter(['Modality [CS:0008,0060]', 'Manufacturers Model Name [LO:0008,1090]',
       'Patients Name [PN:0010,0010]', 'Patient ID [LO:0010,0020]',
       'Patients Sex [CS:0010,0040]', 'Patients Age [AS:0010,1010]',
       'Patients Size [DS:0010,1020]', 'Patients Weight [DS:0010,1030]',
       'Body Part Examined [CS:0018,0015]', 'Scan Options [CS:0018,0022]',
       'Slice Thickness [DS:0018,0050]', 'KVP [DS:0018,0060]',
       'Table Feed per Rotation [FD:0018,9310]',
       'Total Collimation Width [FD:0018,9307]', 'CTDIvol [FD:0018,9345]',
       'X-Ray Tube Current [IS:0018,1151]', 'AcquisitionTime [TM:0008,0032]',
       'Contrast/Bolus Agent [LO:0018,0010]', 'Slice Location [DS:0020,1041]'])

In [6]:
newcolumns = {'Modality [CS:0008,0060]':'Modality', 'Manufacturers Model Name [LO:0008,1090]':'Manufacturer',
       'Patients Name [PN:0010,0010]':'PatientName', 'Patient ID [LO:0010,0020]':'PatientID',
       'Patients Sex [CS:0010,0040]':'Sex', 'Patients Age [AS:0010,1010]':'Age',
       'Patients Size [DS:0010,1020]':'Size', 'Patients Weight [DS:0010,1030]':'Weight',
       'Body Part Examined [CS:0018,0015]':'BodyPart', 'Scan Options [CS:0018,0022]':'ScanMode',
       'Slice Thickness [DS:0018,0050]':'SliceThickness', 'KVP [DS:0018,0060]':'kVp',
       'Table Feed per Rotation [FD:0018,9310]':'TableFeedperRotation',
       'Total Collimation Width [FD:0018,9307]':'Collimation', 'CTDIvol [FD:0018,9345]':'CTDIvol',
       'X-Ray Tube Current [IS:0018,1151]':'mA', 'AcquisitionTime [TM:0008,0032]':'AcquisitionTime',
       'Contrast/Bolus Agent [LO:0018,0010]':'Contrast', 'Slice Location [DS:0020,1041]':'Location'}

In [18]:
data2 = data.rename(columns=newcolumns)
data2.head()

,Modality,Manufacturer,PatientName,PatientID,Sex,Age,Size,Weight,BodyPart,ScanMode,SliceThickness,kVp,TableFeedperRotation,Collimation,CTDIvol,mA,AcquisitionTime,Contrast,Location
0,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,55.5
1,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,64.5
2,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,154.5
3,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,155.5
4,CT,Brilliance 10,Anonymous,NaN,F,054Y,NaN,NaN,NaN,HELIX,2.0,120,NaN,NaN,18.0,405,132934.0,NaN,156.5


**Vamos descobrir os parâmetros dosimétricos de um exame?**
(Como o dataframe é pequeno filtrei por Manufacturer-age para achar o paciente, porém no ambiente hospitalar podemos filtrar por ID)

In [21]:
filtropaciente = data2[(data2.Manufacturer=="LightSpeed16") & (data2.Age=="080Y") ]
filtropaciente

,Modality,Manufacturer,PatientName,PatientID,Sex,Age,Size,Weight,BodyPart,ScanMode,SliceThickness,kVp,TableFeedperRotation,Collimation,CTDIvol,mA,AcquisitionTime,Contrast,Location
9970,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,337,145030.0,NaN,24.50
9971,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,336,145030.0,NaN,13.25
9972,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,249,145030.0,NaN,-99.25
9973,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,251,145030.0,NaN,-100.50
9974,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,252,145030.0,NaN,-101.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10206,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,240,145030.0,NaN,-93.00
10207,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,243,145030.0,NaN,-94.25
10208,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,244,145030.0,NaN,-95.50
10209,CT,LightSpeed16,Anonymous,NaN,M,080Y,NaN,NaN,NaN,HELICAL MODE,1.25,120,27.499998,20.0,NaN,246,145030.0,NaN,-96.75


In [38]:
#Ainda não temos o loadingtube nesta célula, temos apenas a corrente total em mA...
mA = filtropaciente.filter(["Manufacturer","mA"])
mATotal = mA.groupby(["Manufacturer"]).sum()
mAtotal

,mA
Manufacturer,
LightSpeed16,81827
